<a href="https://colab.research.google.com/github/dkplo/code/blob/master/Encripta%C3%A7%C3%A3o_vetoriza%C3%A7%C3%A3o_ELGAMAL_chave_e_k_fixo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time

In [2]:
temp_inicial = time.asctime()
print(temp_inicial)

Sun Sep  6 11:57:18 2020


In [3]:
pip install PyDrive

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 


In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
downloaded = drive.CreateFile({'id':"1D0Rv2TsRIqc6R-FHqk64UGhlWk_f55wO"})
downloaded.GetContentFile('good.csv')

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('good.csv')

In [9]:
data = data.drop(data.columns[1:], axis=1)

In [10]:
data.head()

,text
0,Nycticebus linglom is a fossil strepsirrhine p...
1,Oryzomys pliocaenicus is a fossil rodent from ...
2,.hack dt hk is a series of single player actio...
3,The You Drive Me Crazy Tour was the second con...
4,0 8 4 is the second episode of the first seaso...


In [11]:
temp_carreg_planilha = time.asctime()
print(temp_carreg_planilha)

Sun Sep  6 11:58:29 2020


In [12]:
data['separado_128'] = 0

In [13]:
for i in data.index:
    a_dividir = data['text'][i]
    #print(tamanho, type(tamanho), a_dividir[0:10], type(a_dividir))
    lista = []
    j = 0
    while j < len(a_dividir):
        lista.append(a_dividir[j:j+127])
        j += 127
    data['separado_128'][i] = lista   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
temp_sep_blocos = time.asctime()
print(temp_sep_blocos)

Sun Sep  6 11:58:32 2020


In [15]:
pip install PyCrypto

     |████████████████████████████████| 450kB 2.6MB/s 
  Created wheel for PyCrypto: filename=pycrypto-2.6.1-cp36-cp36m-linux_x86_64.whl size=499939 sha256=ce42f8b28b9fdd7fe3cb6ae019508ba33912dd9de68d4763775bf2f647a66b20
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built PyCrypto


In [16]:
from Crypto import Random
import random

In [17]:
data['criptografia_elgamal_parte1'] = 0
data['criptografia_elgamal_parte2'] = 0
#data['criptografia_elgamal_parte1_num'] = 0
#data['criptografia_elgamal_parte2_num'] = 0

In [18]:
from Crypto.PublicKey import ElGamal

In [19]:
random_generator = Random.new().read
key_elgamal = ElGamal.generate(1024,random_generator)
elgamal_k = random.getrandbits(16)
#key_elgamal_gerada = ElGamal.construct((9967,3543,6056,207)) #construção de uma chave com os parâmetros P, G, Y, X.

In [30]:
for j in data.index:
  lista = data['separado_128'][j]
  lista_criptografada_elgamal_parte1 = []
  lista_criptografada_elgamal_parte2 = []
  #lista_criptografada_elgamal_parte1_num = []
  #lista_criptografada_elgamal_parte2_num = []
  for i in range(0,len(lista)):
    a_criptografar = lista[i]
    lista_criptografada_elgamal_parte1.append(key_elgamal.encrypt(bytes(a_criptografar, encoding="ascii"),elgamal_k)[0])
    lista_criptografada_elgamal_parte2.append(key_elgamal.encrypt(bytes(a_criptografar, encoding="ascii"),elgamal_k)[1])
    #lista_criptografada_elgamal_parte1_num.append(int.from_bytes(lista_criptografada_elgamal_parte1[i], byteorder="little"))
    #lista_criptografada_elgamal_parte2_num.append(int.from_bytes(lista_criptografada_elgamal_parte2[i], byteorder="little"))
  data['criptografia_elgamal_parte1'][j] = lista_criptografada_elgamal_parte1
  data['criptografia_elgamal_parte2'][j] = lista_criptografada_elgamal_parte2
  #data['criptografia_elgamal_parte1_num'][j] = lista_criptografada_elgamal_parte1_num
  #data['criptografia_elgamal_parte2_num'][j] = lista_criptografada_elgamal_parte2_num

In [31]:
data['criptografia_elgamal_parte1'][1][0] == data['criptografia_elgamal_parte1'][2][0]

True

In [32]:
temp_elgamal = time.asctime()
print(temp_elgamal)

Sun Sep  6 12:38:51 2020


In [33]:
lista_hex =[]
for i in range(0,2**(8)):
    lista_hex.append(hex(i))

In [34]:
data['vetor'] = 0

In [35]:
data['vetor'] = data['vetor'].astype(object)

In [37]:
for i in data.index:
  vetor_mapeamento = {}
  for q in lista_hex:
    vetor_mapeamento[q] = 0
  lista_recuperada = data['criptografia_elgamal_parte1'][i][0]
  for j in range(0,len(lista_recuperada)):
    a_mapear = bytes(lista_recuperada[j])
    for w in a_mapear:
      vetor_mapeamento[hex(w)] += 1
  lista_recuperada = data['criptografia_elgamal_parte2'][i]
  for j in range(0,len(lista_recuperada)):
    a_mapear = lista_recuperada[j]
    for w in a_mapear:
      vetor_mapeamento[hex(w)] += 1
  data['vetor'][i] = list(vetor_mapeamento.values())

In [38]:
data['label'] = 8

In [39]:
from google.colab import drive

In [40]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [41]:
from numpy import save

In [42]:
save('/content/drive/My Drive/elgamal_chave_1024_06_09_20_chave_e_k_fixos.npy',data.loc[:, 'vetor':'label'].to_numpy())

In [43]:
data.loc[:, 'vetor':'label'].to_csv('elgamal_chave_1024_06_09_20_chave_e_k_fixos.csv', index = True)
!cp elgamal_chave_1024_06_09_20_chave_e_k_fixos.csv "/content/drive/My Drive/"

In [ ]:
#data.loc[:, 'criptografia_elgamal_parte1':'criptografia_elgamal_parte2'].to_csv('criptogramas_elgamal_chave_1024_31_08_20.csv', index = False)
#!cp criptogramas_elgamal_chave_1024_31_08_20.csv "/content/drive/My Drive/"